<a href="https://colab.research.google.com/github/yj1008/Share_FinancialSMS/blob/master/Success_Show_Data_Korean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
data = pd.read_csv(r'C:\Users\yoonj\OneDrive\Documents\KB_Data_TextSmishing\Original\train_ansi.csv',engine='python')

In [0]:
data[:15]

# 정제

In [0]:
del data['id']
del data['year_month']
data.drop_duplicates("text")
data['text'] = data['text'].str.strip()
data['text2'] = data['text'].str.replace(pat="X", repl="", regex=False)
del data['text']

In [0]:
testtext = data['text2'][9]

# 스미싱 데이터만 가져와서 단어 빈도수 계산

In [0]:
sample_smishing = data[data.smishing.isin([1])].head()
text2 = sample_smishing['text2']

In [0]:
print(text2)

9     (광고)Ba고객님들 뒤엔 언제나 새로운 마음가짐으로 새롭게 준비합니다.당행상품의 자...
23    (광고)추가 품   특판 안내문 지점에서 취급하고 있는 여신  드림론과 달리 여신영...
31    (광고)신용관리  알고 싶다나의 신용과 재무상태는 직접 관리해야지 누군가가 대신해주...
40    (광고)안녕하세요수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA ...
50    (광고)한국citi bank 나의 대출한도와 금리는? 대출때문에 고민하고 있다거나 ...
Name: text2, dtype: object


# 불용어가져오기

In [0]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [0]:
stopw = pd.read_csv(r'C:\Users\yoonj\OneDrive\Documents\KB_Data_TextSmishing\한국어불용어100.txt',engine='python')
stopw.columns = ["s"]

In [0]:
stopw["s"] = stopw["s"].str.split('\t').str[0]
stopw = stopw["s"].tolist()
print(stopw)

['있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓', '을', '어', '에', '시', '드리', '는', '으로', 'ㄴ', '습니다', 'ㅂ니다', '고', '를', '분', 'ㄹ', '면', '겠', '아', '만', '과', '로', '에서', '게', '도']


In [0]:
from konlpy.tag import Kkma
kkma = Kkma()

text_tokens = []

for s in text2:
    text_m = kkma.morphs(s)
    text_tokens.append(text_m)
words_kkma=sum(text_tokens, [])
print(words_kkma)

In [0]:
result = []
for w in words_kkma:
    if w not in stopw:
        result.append(w)
words_kkma = result
print(words_kkma)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
tokenizer = Tokenizer()

In [0]:
tokenizer.fit_on_texts(words_kkma) #빈도수 기준으로 단어 집합 생성

print(tokenizer.word_index) #인덱싱 포함
print("\n")
print(tokenizer.word_counts) #빈도수

In [0]:
from konlpy.tag import Okt
okt = Okt()

text_tokens = []

for s in text2:
    text_m = okt.morphs(s)
    text_tokens.append(text_m)
words_okt=sum(text_tokens, [])
print(words_okt)

['(', '광고', ')', 'Ba', '고객', '님', '들', '뒤', '엔', '언제나', '새로운', '마음가짐', '으로', '새롭게', '준비', '합니다', '.', '당', '행', '상품', '의', '자격', '기준', '과', '심사', '기준', '이', '완화', '되어', '상품', '에', '대해', '간단하게', '상품', '정보', '전달', '드립니다', '.', '수신', '을', '희망', '하지', '않으실', '경우', '에는', '거부', '라는', '답장', '을', '주시', '면', 'KISA', '수신', '거부', '목록', '에', '등록', '을', '시켜', '두', '번', '다시', '발송', '되지', '않도록', '조치', '를', '취', '해드리겠습니다', '.', '신청', '방법', ':', '--', '터치', '하셔서', '문자', '로', '상담', '또는', '00시', '00분', '상담', '이라고', '답장', '을', '보내주시거나', '전화', '를', '주시', '면', '친절하고', '안전한', '상담', '으로', '도움', '드리겠습니다', '.', '카카오', '톡', '상담', '서비스', '시행', '중', 'citibankloan', '친구', '추가', '후', '상담', '신청', '이런', '분', '들', '께', '권', '해드립니다', '.', '1', '.', '시중', '은행권', '에서', '기존', '한도', '를', '모두', '사용', '중', '이신', '분', '.', '2', '.', '기', '대출', '의', '원리금', '균등', '방식', '으로', '인해', '월', '불입', '금액', '이', '부담스러우신', '분', '.', '3', '.', '총', '채무', '금액', '은', '작으나', '채', '무건수가', '많아서', '관리', '가', '안', '되시는', '고객', '님', '들께', '말씀드립니다',

In [0]:
result = []
for w in words_okt:
    if w not in stopw:
        result.append(w)
words_okt = result
print(words_okt)

['(', '광고', ')', 'Ba', '고객', '님', '뒤', '엔', '언제나', '새로운', '마음가짐', '새롭게', '준비', '합니다', '.', '당', '행', '상품', '의', '자격', '기준', '심사', '기준', '완화', '되어', '상품', '대해', '간단하게', '상품', '정보', '전달', '드립니다', '.', '수신', '희망', '하지', '않으실', '에는', '거부', '라는', '답장', '주시', 'KISA', '수신', '거부', '목록', '등록', '시켜', '발송', '되지', '않도록', '조치', '취', '해드리겠습니다', '.', '신청', '방법', ':', '--', '터치', '하셔서', '문자', '상담', '또는', '00시', '00분', '상담', '이라고', '답장', '보내주시거나', '전화', '주시', '친절하고', '안전한', '상담', '도움', '드리겠습니다', '.', '카카오', '톡', '상담', '서비스', '시행', 'citibankloan', '친구', '추가', '후', '상담', '신청', '께', '권', '해드립니다', '.', '1', '.', '시중', '은행권', '기존', '한도', '모두', '사용', '이신', '.', '2', '.', '기', '대출', '의', '원리금', '균등', '방식', '인해', '불입', '금액', '부담스러우신', '.', '3', '.', '총', '채무', '금액', '은', '작으나', '채', '무건수가', '많아서', '관리', '되시는', '고객', '님', '들께', '말씀드립니다', '.', '높은', '금리', '힘들다고', '느끼셨을', '대출', '필요하실', '언제나', '고객', '님', '뒤', '든든한', '파트너', '로서', '와', '드리겠습니다', '.', '당', '행', '상품', '특징', '1', '.', '당', '행', '거래', '없으셔도', '진행', '가능'

In [0]:
tokenizer.fit_on_texts(words_okt) #빈도수 기준으로 단어 집합 생성

print(tokenizer.word_index) #인덱싱 포함
print("\n")
print(tokenizer.word_counts) #빈도수

{'상품': 1, '상담': 2, '대출': 3, '한도': 4, '금리': 5, '2': 6, '부채': 7, '등급': 8, '의': 9, '가능': 10, '까지': 11, '주시': 12, '신용': 13, '통합': 14, '1': 15, '광고': 16, '문자': 17, '드리겠습니다': 18, '추가': 19, '3': 20, '자체': 21, '고객': 22, '자격': 23, '신청': 24, '와': 25, '은행': 26, '금융': 27, '수신': 28, '거부': 29, '답장': 30, '방법': 31, '전화': 32, '기존': 33, '관리': 34, '따라': 35, '차등': 36, '적용': 37, '님': 38, '행': 39, '진행': 40, '전환': 41, '연봉': 42, '당': 43, '희망': 44, '도움': 45, '권': 46, '원리금': 47, '균등': 48, '방식': 49, '채무': 50, '높은': 51, '판단': 52, '조회': 53, '직장인': 54, '최대': 55, '최저': 56, '저': 57, '해드리겠습니다': 58, '카카오': 59, '톡': 60, '서비스': 61, '후': 62, '시중': 63, '은행권': 64, '이신': 65, '은': 66, '4': 67, '이자': 68, '4000만': 69, '97': 70, '기업': 71, '리스트': 72, '대비': 73, '하고': 74, '판매': 75, '해': 76, '자금': 77, '입니다': 78, '주시거나': 79, '연락': 80, '기준': 81, '되어': 82, '시켜': 83, '발송': 84, '이라고': 85, '친절하고': 86, '안전한': 87, '시행': 88, '친구': 89, '해드립니다': 90, '금액': 91, '되시는': 92, '보다': 93, '자': 94, '및': 95, '안내': 96, '1억': 97, '300': 98, '새': 99, '8': 1

In [0]:
#text_token = word_tokenize(testtext)
#from nltk.tag import pos_tag
#text_token_pos = pos_tag(text_token)

# Word2Vec

In [0]:
from gensim.models import Word2Vec
model = Word2Vec(words_okt, size=100, window=5, min_count=5, workers=4, sg=0)

# 케라스를 이용한 원핫인코딩

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [0]:
result = []
for x in text2:
    encoded=tokenizer.texts_to_sequences([x])[0]
    one_hot = to_categorical(encoded)
    result.append(one_hot)
    
print(result)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0

In [0]:
encoded=tokenizer.texts_to_sequences([words_okt[0]])[0]
print(encoded)
#one_hot = to_categorical(encoded)
#result.append(one_hot)